In [17]:
# Imports and loading data
import json
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

data_file = open('goemotions.json')
data = json.load(data_file)

In [75]:
# Collecting the words and their frequencies (2.1)
tokens = []
y_emotions = []
y_sentiments = []

for lists in data:
    y_emotions.append(lists[1])
    y_sentiments.append(lists[2])
    for text in lists[:1]:
        tokens.append(text)
        
cv = CountVectorizer(analyzer='word')
        
X = cv.fit_transform(tokens)

In [76]:
print("Vocabulary: ", cv.vocabulary_)

Vocabulary:  {'that': 26899, 'game': 11309, 'hurt': 13362, 'you': 30300, 'do': 8256, 'right': 22728, 'if': 13530, 'don': 8337, 'care': 4571, 'then': 26939, 'fuck': 11116, 'em': 9069, 'man': 16377, 'love': 16094, 'reddit': 21940, 'name': 17815, 'was': 29324, 'nowhere': 18409, 'near': 17930, 'them': 26929, 'he': 12616, 'by': 4307, 'the': 26906, 'falcon': 9976, 'considering': 6087, 'it': 14466, 'such': 26023, 'an': 1612, 'important': 13703, 'document': 8268, 'should': 24177, 'know': 15107, 'damned': 7006, 'thing': 26995, 'backwards': 2581, 'and': 1652, 'forwards': 10888, 'thanks': 26891, 'again': 1212, 'for': 10773, 'help': 12745, 'isn': 14445, 'as': 2099, 'big': 3218, 'but': 4261, 'still': 25658, 'quite': 21477, 'popular': 20482, 've': 28825, 'heard': 12653, 'same': 23216, 'about': 810, 'his': 12924, 'content': 6156, 'never': 18073, 'watched': 29353, 'him': 12893, 'much': 17629, 'crazy': 6556, 'went': 29528, 'to': 27300, 'super': 26131, 'religion': 22202, 'high': 12842, 'school': 23447, 

In [77]:
#Printing the size of the vocabulary (2.1)
print(len(cv.vocabulary_))

30449


In [103]:
# Splitting the data in two sets (2.2)
X_training, X_testing, y_training_emotions, y_testing_emotions = train_test_split(X, y_emotions, test_size=0.2, train_size=0.8, shuffle=False)

In [105]:
# Printing sample of training data
print(X_training.shape)
print(len(y_training_emotions))

(137456, 30449)
137456


In [94]:
# Printing sample of test data
print(test_set.shape)

(34364, 30449)


In [102]:
from sklearn.naive_bayes import MultinomialNB

multinomial_naive_bayes_emotions = MultinomialNB()
multinomial_naive_bayes_emotions.fit(X[0], np.array(y_emotions))

ValueError: Found input variables with inconsistent numbers of samples: [1, 171820]

In [85]:
print(multinomial_naive_bayes_emotions.predict(X[2:3]))

['love']
